# Experiment Results

## Details

- generation of inputs with [tribble](https://github.com/havrikov/tribble/)
- inputs generated from two grammars, one based on the current [URL standard](https://url.spec.whatwg.org/), the other one based on the RFC documents [RFC 3986](https://tools.ietf.org/html/rfc3986#appendix-A) and [RFC 6874](https://tools.ietf.org/html/rfc6874)
- experiment executed for both grammars with identical docker images
- for each run: 10 inputs selected at random from the generated inputs and added to the tests to execute
- run x contains all inputs used in runs 0..x-1 and 10 newly selected inputs
- execution of tests for 11 URL parsers:


*Language* | *URL Parser* | *Coverage Tool* 
--- | --- | ---
Firefox | nsURLParsers.cpp | grcov/genhtml
Chromium | url_parse.cc | coverage.py script
C | uriparser | LCOV 
C++ | POCO |LCOV 
Go | Package net/url | Package testing with coverage flags 
Java | java.net URL Class | JCOV 
JavaScript <br> | urijs <br> jsdom/whatwg-url| nyc/istanbul <br>
PHP | League URI | PHPUnit  
Python 3| urllib.parse | Coverage.py  
Ruby | Module URI |SimpleCov 

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install pandas markdown matplotlib
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import random
from IPython.display import display, Markdown


# view results of 100 runs with single inputs added at "./smallExp/(ls | rfc)"
ls_df=pd.read_csv("./ls/experimentResultsMain.csv", index_col='run_nr').rename(columns=str.lower)

ls_df_comp=pd.read_csv("./ls/experimentResultsComponents.csv", index_col='run_nr').rename(columns=str.lower)

rfc_df=pd.read_csv("./rfc/experimentResultsMain.csv", index_col='run_nr').rename(columns=str.lower)
rfc_df_comp=pd.read_csv("./rfc/experimentResultsComponents.csv", index_col='run_nr').rename(columns=str.lower)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


parsers=[c[:-4] for c in ls_df.columns.values if "-cov" in c]
colors_all = ['orange', 'darkblue', 'red', 'yellow', 'lightblue', 'purple', 'beige', 'grey', 'black', 'pink','green']
colors_map={}
chars = '0123456789ABCDEF'
# coloring
i=0
for p in parsers:
    c=colors_all[i] 
    colors_map[p]=c
    
    i+=1

comp_colors={}
for col in ls_df_comp.columns:
    if '_success-' in col:
        comp_colors[col]='#0'+''.join(random.sample(chars,5))
    elif '_success' in col:
        comp_colors[col]='#8'+''.join(random.sample(chars,5))
    else:
        comp_colors[col]='#F'+''.join(random.sample(chars,5))
for col in rfc_df_comp.columns:
    if col not in comp_colors:
        comp_colors[col]='#7'+''.join(random.sample(chars,5))
    
    
colors_comp = ['#2fb553', '#4a76bd']
ms=22

def setTickDistances(n, axs):
    ticks = axs.xaxis.get_ticklocs()
    ticklabels = [l.get_text() for l in axs.xaxis.get_ticklabels()]
    axs.xaxis.set_ticks(ticks[::n])
    axs.xaxis.set_ticklabels(ticklabels[::n])
    return


The number of inputs added in each run (10) is large enough to reach the maximum coverage in the initial run for some parsers. To ensure that this is not caused by the test execution, an additional experiment in which single inputs are added was conducted. Results of this experiment can be viewed by using `./smallExp/ls` and `./smallExp/rfc` when loading csv files.





## Living Standard Results
### Tabular View

In [ ]:
ls_df.head(10)

In [ ]:
ls_df.tail(10)

### Overview Plot 

In [ ]:
def overviewPlot(df, title, xrange=None):
    fig, axs=plt.subplots(figsize=(40,20))
    plt.rcParams['font.size']='40'

    # apply some nice spacing
    if xrange is not None:
        a,b=xrange
        title+=" from run "+str(a)+" to run "+str(b)+" \n"
        xrange =(a-0.7, b+0.7)
    
    for p in parsers:
        plot=df[p+"-cov"].plot(title=title,
                       ylim=(0,100),xlim=xrange, ax=axs,
                       color=colors_map[p], style=".", ms=2*ms,
                      label=p.capitalize(), fontsize=40)
    


    plot.legend(bbox_to_anchor=(1,-0.2),markerscale=2.)
    plot.grid(True)

    axs.set_ylabel("Coverages", fontsize=40)
    axs.set_xlabel("Run", fontsize=40)
    
    vals = axs.get_yticks()
    axs.set_yticklabels(['{:.0f}'.format(x)+"%" for x in vals])
    
    #plt.show()
    plt.savefig('test.pdf')
    plt.show()
    
    display(Markdown("When directly comparing the coverages achieved in different parsers, one should always consider \
                    the number of lines coverable for each parser by simply parsing inputs. The coverages depicted here \
                    are calculated in relation to the total number of lines in each parser."))

In [ ]:
overviewPlot(ls_df,"Living Standard Coverages Overview\n", )

# plotting only selected runs
#overviewPlot(ls_df,"Living Standard Coverages Overview\n", (0,10) )
#overviewPlot(ls_df,"Living Standard Coverages Overview\n", (100,110) )

In [ ]:
#TODO include some details about the parsers here

### Errors and Exceptions

In [ ]:
def errorsOverviewPlot(df, title):
    plt.rcParams['font.size']='30'
    fig, axs=plt.subplots(figsize=(20,15))
    
    new_df=pd.DataFrame()
    err_df=pd.DataFrame()

    print(df.tail(1)['nr-inputs'])
    for c in df.columns:
        if 'exceptions' in c:
            new_df[c]=df[c]
            err_df[c]=(df[c]/df['nr-inputs'])*100
        
    

    plot=df.tail(1).plot(kind='barh', title=title,
                       ax=axs, width=1.9,
                       y=[ cn for cn in df.columns if 'exceptions' in cn],
                       color=colors_map.values(), rot=0) 
    
    plot.legend(markerscale=2., loc=(1.2,0))
    plot.grid(True)


    for bar in plot.patches:
        plot.annotate(format(bar.get_width(), '.0f'), 
                    (bar.get_width() ,bar.get_y() + bar.get_height() / 3 ),
                    ha='left',  xytext=(5, 0), 
                   textcoords='offset points', color='dimgrey')


    
    axs.set_ylabel("Run", fontsize=40)
    axs.set_xlabel("\nExceptions", fontsize=40)
    axs.set_axisbelow(True)
    
    plt.show()
    display(Markdown("The above diagram compares the number of parsing exceptions encountered for each parser in \
                    the last available run with "+str(df['nr-inputs'].max()) +" distinct inputs."))
    
    display(new_df.tail(1).transpose())
    display(err_df.tail(1).transpose())
    return err_df
    
def errorsOverTime(df, title, xrange=None):
    plt.rcParams['font.size']='40'
    fig2, axs2=plt.subplots(figsize=(50,20))
    
    # apply some nice spacing
    if xrange is not None:
        a,b=xrange
        title+=" from run "+str(a)+" to run "+str(b)+" \n"
        xrange =(a-0.7, b+0.7)
    
    
    for p in parsers:
        plot2=df[p+"-exceptions"].plot(title=title,
                        ax=axs2,xlim=xrange,
                       color=colors_map[p], style=".", ms=ms,
                      label=p.capitalize(), fontsize=40)
    plot2.grid(True)
    plot2.legend(markerscale=2., loc='best')
    axs2.set_ylabel("Exceptions\n", fontsize=40)
    axs2.set_xlabel("\nRun", fontsize=40)
    axs2.set_axisbelow(True)
    plt.show()

In [ ]:

errorsOverviewPlot(ls_df, "Living Standard Exceptions Overview\n")
errorsOverTime(ls_df, "Exceptions over time\n")
# errors within a specific range
# errorsOverTime(ls_df, "Exceptions over time\n", (0,100))


### Equal and Unequal Results

In [ ]:
def equalResultsOverview(df, title):
    fig, axs=plt.subplots(figsize=(30,15))
    plt.rcParams['font.size']='30'

    new_df=pd.DataFrame()
    b_new_df=pd.DataFrame()
    for c in df.columns:
        if 'result' in c:
            if 'b-' in c:
                b_new_df[c]=df[c]
            else:
                new_df[c]=df[c]
    
    
    plot=new_df.iloc[::df['nr-inputs'].max()//100].plot(kind='bar', stacked=True, ax=axs, rot=0,
                     color=['g', 'pink', 'r'], title=title)
    
    for container, hatch in zip(plot.containers, ("/", ".")):
        for patch in container.patches:
            patch.set_hatch(hatch)
    
    n=df['nr-inputs'].max()/100
    #setTickDistances(int(n), axs)
    
    plot.grid(True)
    plot.legend(loc='best')
    axs.set_ylabel("Inputs", fontsize=40)
    axs.set_axisbelow(True)
    plt.show()
    display(Markdown("In the above diagram a ``eq-success-result`` means that all considered parsers successfully parsed \
    the input. A ``eq-fail-result`` means that all parsers rejected the given input and a ``neq-result`` means that \
    some parsers accepted the input as valid while other parsers rejected the input."))
    return b_new_df

In [ ]:
ls_b_new_df=equalResultsOverview(ls_df, "Living Standard Parser Equality\n")

In [ ]:
def browserEquality(df, title):
    fig, axs=plt.subplots(figsize=(30,15))
    plt.rcParams['font.size']='30'
    nr_inputs=sum([df[col].tail(1) for col in df.columns if 'result' in col])
    n=int(nr_inputs)//100
    plot=df.iloc[::n].plot(kind='bar', stacked=True, rot=0, ax=axs, 
            color=['g', 'pink', 'orange', 'lightblue', 'r'], title=title)
    axs.set_ylabel("Inputs", fontsize=40)
    plot.grid(True)
    axs.set_axisbelow(True)
    for container, hatch in zip(plot.containers, ("/", ".", "\\", "x")):
        for patch in container.patches:
            patch.set_hatch(hatch)
    
    plot.grid(True)
    plot.legend(loc='best')
    plt.show()
    

In [ ]:
browserEquality(ls_b_new_df, "Living Standard Browser Results Equality\n")
display(Markdown("Much like the previous diagram, this diagram differs between equal and unequal parsing results. \
                    Because only the results of the tested browsers are considered, the diagram can also show whether \
                    the browsers failed on the same components, on different components, or if the results were completely \
                    unequal i.e. one rejection and one component failure."))

#### Browser Components

In [ ]:
def browserComponentComparison(df_comp, title):
    fig, axs=plt.subplots(figsize=(45,25))
    plt.rcParams['font.size']='30'

    new_df=pd.DataFrame()
    for c in df_comp.columns:
        if 'nr' not in c:
            new_df[c]=df_comp[c]
    
    colors=[comp_colors[name] for name in new_df.columns]
    nr_inputs=sum([new_df[col].tail(1) for col in new_df.columns])
    n=int(nr_inputs)//100
    plot=new_df.iloc[::n].plot(kind='bar', stacked=True, ax=axs, rot=0, color=colors ,
                     title=title)
    axs.set_ylabel("Inputs", fontsize=40)
    
    plot.grid(True)
    
    try:
        plot.legend(loc=(0,-0.4), ncol=len(new_df.columns)//5)
    except:
        plot.legend(loc='best')
    axs.set_axisbelow(True)
    plt.show()
    display(Markdown("The above diagram shows in detail which combination of results was \
            observed for the considered browsers. The following table shows the combination \
            of results of the last available run in numerical form."))
    
    
    trdf=new_df.tail(1).transpose()
    #trdf=new_df.head(1000).tail(1).transpose()
    display(trdf.sort_values([c for c in trdf.columns], ascending=False).style.format('{:.0f}'))

In [ ]:
absolute_inputs=ls_df_comp.tail(1)['nr_inputs']


browserComponentComparison(ls_df_comp, "Living Standard Browser Component Equality\n")




#### Separately for each Browser

Considering the latest available run for each figure.

In [ ]:
def browserComponentDetail(browser, df, run_nr=-1, abs_inputs=-1, 
                           components=['scheme', 'username', 'password', 'host', 'port', 'path', 'query', 'fragment']):
    
    # select the specified run, default: last run
    if run_nr < 0 :
        df=df.tail(1)
    else:
        df=df.head(run_nr+1).tail(1)
    plt.rcParams['font.size']='20'
    fig, axs=plt.subplots( figsize=(25,15))
    
    
    full_df=pd.DataFrame()
    for component in components:
        # count the relevant columns
        if abs_inputs>0:
            inputs=abs_inputs #use exact nr of unique inputs
        else:
            inputs=int(df['nr_inputs'])
        new_df=pd.DataFrame()
        
        sumcomponent=0
        for c in df.columns:
            if browser+'_'+component in c:
                new_df[c]=df[c]
                sumcomponent+=int(df[c])
        
        new_df['component']=[component]
        new_df['other']=[inputs-sumcomponent]
        full_df=pd.concat([full_df, new_df], ignore_index=True, sort=True)
    
    
    comp_colors2=comp_colors
    comp_colors2['other']='dimgrey'
    
    colors=[comp_colors[name] for name in full_df.columns if name != 'component']
    
    plot=full_df.plot( kind='bar', stacked=True, ax=axs,x='component',
                         color=colors,
                         rot=0, title=browser.capitalize()+" Details\n")
    plt.show()
    
    full_df=full_df.set_index('component')
    format_dict={}
    for cl in full_df.select_dtypes(float).columns:
        format_dict[cl]='{:.0f}'
    display(full_df.head(10).style.format(format_dict))
    display(Markdown('Each table row shows which result combinations occured when the given browser had a component \
            mismatch for the specified component. The column \'other\' represents all results where the given browser \
            did not have a component mismatch for the specified component.'))
    
    
    #calculate percentages
    trdf=full_df.transpose()
    if abs_inputs>0:
        absolute_inputs=abs_inputs
    else:
        absolute_inputs=df.tail(1)['nr_inputs']
    print(absolute_inputs)
    
    err_p_df=pd.DataFrame()
    for c in trdf.columns:
        p= 1-(trdf[c]['other']/absolute_inputs)
        err_p_df[c]=[p]
        
    err_p_df=err_p_df.rename(index={0:browser})
    
    return err_p_df.transpose() #used for table creation
    

View the full results of this run [here](./ls/lastRun/resultOverview.html).

The following graphs show which parsing result the other browser had when one browser had a component mismatch. 

In [ ]:

browserComponentDetail('firefox', ls_df_comp, abs_inputs=int(ls_df.tail(1)['nr-inputs']))
browserComponentDetail('chromium', ls_df_comp,abs_inputs=int(ls_df.tail(1)['nr-inputs']))




In [ ]:
def create_comp_table(comp_df, b_df, steps=100):
    tab_df=comp_df
    col=b_df['nr-inputs']
    tab_df['nr_inputs_unique']=col
    
    tab_df=tab_df.reindex(sorted(tab_df.columns), axis=1)
    last_elem=tab_df.tail(1)
    tab_df=tab_df[::steps].append(last_elem).transpose()

    f='r'
    for c in tab_df.columns:
        f+='r'
    return tab_df.to_latex(column_format=f, float_format="{:.0f}".format)

In [ ]:
print(create_comp_table(rfc_df_comp, rfc_df, 100))

In [ ]:
print(create_comp_table(ls_df_comp, ls_df, 100))

In [ ]:
tab_df=ls_df_comp
col=ls_df['nr-inputs']
tab_df['nr_inputs_unique']=col
tab_df=tab_df.reindex(sorted(tab_df.columns), axis=1)
tab_df=tab_df[::100].transpose()

f='r'
for c in tab_df.columns:
    f+='r'
print(tab_df.to_latex(column_format=f, float_format="{:.0f}".format))

 These graphs only include results where at least one browser had a component mismatch.
Further below are graphs where at least one browser successfully parsed or immediately rejected the input.

#### Parsing Success and Rejection

A parsing success means that the parser accepted the given input as valid URL and all URL components had the expected content. A rejection means that the parser did not accept the given input as valid URL and no component checks were performed. Between these outcomes lies a third possible outcome: the parser accepts the given input as valid URL but at least one component did not have the expected content.

The following figures show which result one browser had when the other browser successfully parsed a URL or when it rejected the URL. 

In [ ]:
def browserDetail(browser, df, run_nr=-1):
    
    # select the specified run, default: last run
    if run_nr < 0 :
        df=df.tail(1)
    else:
        df=df.head(run_nr+1).tail(1)
    plt.rcParams['font.size']='10'
    plt.rcParams['figure.figsize'] = (16,7)
    fig, axs=plt.subplots(1,2)
    #fig2, axs2=plt.subplots( figsize=(8,8))
    
    full_df=pd.DataFrame()
    for component in ['success', 'reject']:
        
        # count the relevant columns
        inputs=int(df['nr_inputs'])
        new_df=pd.DataFrame()
        
        sumcomponent=0
        for c in df.columns:
            if browser+'_'+component in c:
                new_df[c]=df[c]
                sumcomponent+=int(df[c])
        
        new_df['result']=[component]
        
        full_df=pd.concat([full_df, new_df], ignore_index=True, sort=True)
        
        
    full_df=full_df.set_index('result')
    display(full_df.style.format('{:.0f}'))
    tr_df=full_df.transpose().sort_values(by=['success', 'reject'])
    plot=tr_df.plot( kind='pie',ax=axs[0], 
                         colormap='Greens', y='success',
                         labels=None,
                         legend=False)
    
    plot.legend(loc=(0.2,-0.2), labels=[c for c in tr_df.transpose().columns if tr_df['success'][c]>0],fontsize=12)
    
    plot2=full_df.transpose().sort_values(by=['reject']).plot( kind='pie',ax=axs[1],
                         colormap='YlOrRd', y='reject',
                         labels=None, legend=False)
    plot2.legend(loc=(0.2,-0.2),labels=[c for c in tr_df.transpose().columns if tr_df['reject'][c]>0], fontsize=12)
    display(Markdown("### <center>"+browser.capitalize()+" Details</center>\n"))
    plt.show()
    display(Markdown("Note that these pie charts **only** show the other browsers results for \
    success or reject results of the specified browser. The different component combinations are \
    discussed above."))

In [ ]:
browserDetail('firefox', ls_df_comp)


In [ ]:
browserDetail('chromium', ls_df_comp)

## RFC Results

### Tabular View

In [ ]:
rfc_df.head(10)

In [ ]:
rfc_df.tail(10)

### Coverages Overview Plot

In [ ]:
overviewPlot(rfc_df,"RFC Coverages Overview\n" )
# plotting only selected runs
#overviewPlot(rfc_df,"RFC Coverages Overview\n", (0,10) )
#overviewPlot(rfc_df,"RFC Coverages Overview\n", (100,110) )

### Errors and Exceptions Overview

In [ ]:
errorsOverviewPlot(rfc_df, "RFC Exceptions Overview\n") 
errorsOverTime(rfc_df, "Exceptions over time\n") 

### Equal and Unequal Results

In [ ]:
rfc_b_new_df=equalResultsOverview(rfc_df, "RFC Results Parser Equality\n")

In [ ]:
browserEquality(rfc_b_new_df, "RFC Browser Results Equality\n") 
display(Markdown("Because component checks are currently only implemented for ``url-fuzzing/grammars/livingstandard-url.scala`` \
            the above diagram only contains entries where no component checks are required. Similarly, the next diagram \
            only contains entries for parsing success or rejection."))

In [ ]:
browserComponentComparison(rfc_df_comp, "RFC Browser Component Equality\n")

### Detailed Browser Comparison

Considering the last available run. View the full results of this run [here](./rfc/lastRun/resultoverview.html).

In [ ]:
browserDetail('firefox', rfc_df_comp)


In [ ]:
browserDetail('chromium', rfc_df_comp)

## Detailed per Parser Comparison of Grammars

In [ ]:
from IPython.display import display, Markdown

ffexcov=74.4 # link to reports or extract from reports
ffwptcov=84.4
chrexcov=83.65
chrwptcov=64.29

source_reports={}
source_reports["chromium"]="chromium/report.html"
source_reports["firefox"]="firefox/nsURLParsers.cpp.gcov.html" 
source_reports["c"]="C/src/UriParse.c.gcov.html"
source_reports["cpp"]="Cpp/src/URI.cpp.gcov.html"
source_reports["go"]="Go/index.html"
source_reports["java"]="Java/java/net/URL.html" 
source_reports["javascripturijs"]="JavaScript/urijs/URI.js.html"
source_reports["javascriptwhatwg-url"]="JavaScript/whatwg-url/whatwg-url/dist/url-state-machine.js.html"
source_reports["php"]="PHP/index.html"
source_reports["python"]="Python/_usr_lib_python3_6_urllib_parse_py.html"
source_reports["ruby"]="Ruby/index.html"



def compareGrammarResults(parser, df1, newcolumn1, df2, newcolumn2, xrange=None):
    display(Markdown("### "+parser.capitalize()))
    plt.rcParams['font.size']='40'
    fig, axs=plt.subplots(figsize=(40,20))
    
    
    title=parser.capitalize()+" Coverages\n"
    # apply some nice spacing
    if xrange is not None:
        a,b=xrange
        title+=" from run "+str(a)+" to run "+str(b)+" \n"
        xrange =(a-0.7, b+0.7)

    new_df=pd.DataFrame()
    new_df[newcolumn1]=df1[parser+"-cov"]
    new_df[newcolumn2]=df2[parser+"-cov"]

    plot=new_df.plot(title=title,ylim=(0,100), xlim=xrange,
                     ax=axs, style=".", color=colors_comp, ms=ms)
    
    plot.grid(True)
    

    axs.set_ylabel("Coverages", fontsize=40)
    axs.set_xlabel("Run", fontsize=40)
    otherTests=""
    
    if parser=='firefox': #TODO: add coverage reports for wpt and existing tests
        plot.axhline(y=ffexcov, color='black', linestyle=':', lw=4, label='Existing Test Files')
        otherTests+="Existing Test Files Coverage: "+str(ffexcov)+"%"
        otherTests+=" [coverage report](./otherTestResults/firefoxExistingTests/firefox/nsURLParsers.cpp.gcov.html)"
        plot.axhline(y=ffwptcov, color='y', linestyle=':', lw=4, label='WPT tests')
        otherTests+="\n\nWeb platform tests Coverage: "+str(ffwptcov)+"%"
        otherTests+=" [coverage report](./otherTestResults/firefoxWPTReports/firefoxWPTFullURLs/nsURLParsers.cpp.gcov.html)"
    if parser=='chromium':
        plot.axhline(y=chrexcov, color='black', linestyle=':', lw=4, label='Existing Test Files')
        otherTests+="Existing Test Files Coverage: "+str(chrexcov)+"%"
        otherTests+=" [coverage report](./otherTestResults/chromiumExistingTests/chromiumExistingFull/report.html)"
        plot.axhline(y=chrwptcov, color='y', linestyle=':', lw=4, label='WPT tests')
        otherTests+="\n\nWeb platform tests Coverage: "+str(chrwptcov)+"%"
        otherTests+=" [coverage report](./otherTestResults/chromiumWPTReports/chromiumFullWPT/report.html)"
    plot.legend(loc='best',markerscale=2.)
    plt.show()
    if xrange is not None:
        display(Markdown("#### Results from run "+str(a)+" to run "+str(b)))
        display(new_df[a:b].describe())
        display(Markdown(newcolumn1+" Max Coverage: "+str(new_df[a:b][newcolumn1].max())+"% \
                     reached in run "+str(new_df[a:b][newcolumn1].idxmax())+" "))
        display(Markdown(newcolumn2+" Max Coverage: "+str(new_df[a:b][newcolumn2].max())+"% \
                     reached in run "+str(new_df[a:b][newcolumn2].idxmax())+" "))
        display(Markdown(otherTests))
    else:
        display(new_df.describe())
        
    display(Markdown("#### Overall max results"))
    display(Markdown(newcolumn1+" Max Coverage: "+str(new_df[newcolumn1].max())+"% \
                     reached in run "+str(new_df[newcolumn1].idxmax())+" \
                     \n\n[full max run results](./ls/"+parser+"/resultoverview.html) \
                     \n\n[coverage report](./ls/"+parser+"/"+source_reports[parser]+")"))
    
    display(Markdown(newcolumn2+" Max Coverage: "+str(new_df[newcolumn2].max())+"% \
                     reached in run "+str(new_df[newcolumn2].idxmax())+"\
                     \n\n[full max run results](./rfc/"+parser+"/resultoverview.html)\
                     \n\n[coverage report](./rfc/"+parser+"/"+source_reports[parser]+")"))
    display(Markdown(otherTests))
    
    return new_df, plot



In [ ]:
dfs={}
plots=[]
for parser in sorted(parsers):
    dfs[parser], plot=compareGrammarResults(parser, ls_df, "Living Standard", rfc_df, "RFC")
    plots+=[plot]
    

# focusing on a range
#for parser in parsers:
#    dfs[parser]=compareGrammarResults(parser, ls_df, "Living Standard", rfc_df, "RFC", (0,10))
    



The cells below are used to create data representations in latex-friendly formatting.

In [ ]:
plt.rcParams['figure.constrained_layout.use'] = True
figure, axs = plt.subplots(6,2, figsize=(40,60))
plot_index=0
axs[-1, -1].axis('off')
for loc in axs.flat:
    try:
        new_df=dfs[parsers[plot_index]]
        plot_index+=1
    except:
        break
    
    plot=new_df.plot(title=parsers[plot_index-1].capitalize()+"\n",ylim=(0,100), 
                     ax=loc, style=".", color=colors_comp, ms=ms/2, legend=False)
    vals = loc.get_yticks()
    loc.set_yticklabels(['{:.0f}'.format(x)+"%" for x in vals])
    plot.grid(True)
    plot.set_xlabel("Run")
    #plot.legend(loc='best',markerscale=2.)


for ax in figure.get_axes():
    if ax != axs[4][1]:
        ax.label_outer()
    else:
        ax.set_yticklabels(['' for x in vals])
        handles, labels = ax.get_legend_handles_labels()
        figure.legend(handles, labels, loc=(0.6,0.1), markerscale=4.)
        
        

plt.savefig('ov_detail.pdf')

    

The number of inputs added in each run (10) is large enough to reach the maximum coverage in the initial run for some parsers. To ensure that this is not caused by the test execution, an additional experiment in which single inputs are added was conducted. Results of this experiment can be viewed by using `./smallExp/ls` and `./smallExp/rfc` when loading csv files at the beginning.



In [ ]:
    
    plt.rcParams['font.size']='40'
    fig, axs=plt.subplots(figsize=(50,20))
    
    overview_df=pd.DataFrame(index=[d for d in dfs], columns=['Living Standard', "RFC"])
    for p in dfs:
        overview_df.loc[p]['Living Standard']=dfs[p].tail(1)['Living Standard'].iloc[0]
        overview_df.loc[p]['RFC']=dfs[p].tail(1)['RFC'].iloc[0]
    overview_df=overview_df.sort_index()  
    plot=overview_df.plot(kind='bar', title="",ylim=(0,100),
                          color=colors_comp, ax=axs, alpha=0.85)
    plot.grid(True)
    
    display(overview_df)
    #axs.set_ylabel("Coverages", fontsize=40)
    vals = axs.get_yticks()
    axs.set_yticklabels(['{:.0f}'.format(x)+"%" for x in vals])
    
    labels = [item.get_text() for item in axs.get_xticklabels()]
    blabels=[]
    for label in labels:
        blabels+=["\n"+label.replace('script', 'script\n').capitalize()]
    axs.set_xticklabels(blabels)
    plt.xticks(rotation=0, horizontalalignment="center")
    axs.set_axisbelow(True)
    
    
    
    for i in plot.patches:
    # get_width pulls left or right; get_y pushes up or down
        plot.text(i.get_x()+i.get_width()/4, i.get_height()-3.5, 
            str(i.get_height())+'%', fontsize=55, rotation=90,
            color='white')
    
    
    plt.savefig('ov_max_bar.pdf')
    plt.show()
    
    overview_df=pd.DataFrame(index=[d for d in dfs], columns=['Living Standard', 'ls max run', "RFC", 'rfc max run'])
    for p in dfs:
        overview_df.loc[p]['Living Standard']=dfs[p].tail(1)['Living Standard'].iloc[0]
        overview_df.loc[p]['ls max run']=dfs[p]['Living Standard'].idxmax()
        overview_df.loc[p]['RFC']=dfs[p].tail(1)['RFC'].iloc[0]
        overview_df.loc[p]['rfc max run']=dfs[p]['RFC'].idxmax()
    
    for c in overview_df.columns:
        overview_df[c]=pd.to_numeric(overview_df[c])
   
    display(overview_df.describe())
    
    
    
    

In [ ]:
display(overview_df.sort_values(by='Living Standard', ascending=False))
display(overview_df.sort_values(by='RFC', ascending=False))
display(overview_df.sort_values(by='ls max run', ascending=False))
display(overview_df.sort_values(by='rfc max run', ascending=False))
    
overview_df=overview_df.sort_values(by='Living Standard', ascending=False)
print(overview_df.to_latex())

In [ ]:
err_df=browserComponentDetail('firefox', ls_df_comp, abs_inputs=int(ls_df.tail(1)['nr-inputs']), 
                             components=['success', 'scheme', 'username', 'password', 'host',
                                         'port', 'path', 'query', 'fragment', 'reject'])
err_df3=browserComponentDetail('firefox', rfc_df_comp, abs_inputs=int(rfc_df.tail(1)['nr-inputs']), 
                             components=['success', 'reject'])
err_df['firefox rfc']=err_df3['firefox']

err_df2=browserComponentDetail('chromium', ls_df_comp,abs_inputs=int(ls_df.tail(1)['nr-inputs']), 
                             components=['success', 'scheme', 'username', 'password', 'host',
                                         'port', 'path', 'query', 'fragment', 'reject'])

err_df4=browserComponentDetail('chromium', rfc_df_comp,abs_inputs=int(rfc_df.tail(1)['nr-inputs']), 
                             components=['success', 'reject'])

err_df['chromium']=err_df2['chromium']
err_df['chromium rfc']=err_df4['chromium']
display(err_df)

inputsls=int(ls_df.tail(1)['nr-inputs'].max())
inputsrfc=int(rfc_df.tail(1)['nr-inputs'].max())
caption='for run \
'+str(ls_df.tail(1).index.max())+' with '+str(inputsls)+'(Living Standard) \
and '+str(inputsrfc)+'(RFC) unique inputs'
print(str(ls_df.tail(1).index.max())+" = "+str(rfc_df.tail(1).index.max()))
print(err_df.to_latex(
    float_format="{:.2%}".format).replace("nan\%", "-"))
print('\caption[Browser component errors]{Browser component errors '+caption+'}')

In [ ]:
e_df=errorsOverviewPlot(ls_df, "Living Standard Exceptions Overview\n")
e_df2=errorsOverviewPlot(rfc_df, "RFC Exceptions Overview\n")

e_f_df=pd.DataFrame()
e_f_df['Living Standard']=e_df.tail(1).transpose()[ls_df.tail(1).index.max()]
e_f_df['RFC']=e_df2.tail(1).transpose()[rfc_df.tail(1).index.max()]
display(e_f_df)
print(str(ls_df.tail(1).index.max())+" = "+str(rfc_df.tail(1).index.max()))
print(e_f_df.sort_values(by='Living Standard').to_latex())
print('\caption[Parser error rates]{Parser error rates '+caption+'}')